In [4]:
import pandas as pd
from scipy.stats import entropy
import pandas as pd
import numpy as np

In [5]:
class DataAnonymizer:
    
    def __init__(self, df, sensitive_columns=None):
        self.df = df.copy()
        
        # Ensure sensitive_columns is a list
        if sensitive_columns is None:
            print('Sensitive Column is empty')
            self.sensitive_columns = []
        elif isinstance(sensitive_columns, str):
            self.sensitive_columns = [sensitive_columns]
        else:
            self.sensitive_columns = sensitive_columns
            
    def k_anonymity(self, sum_column, k):
        self.df['K_Anonymity_Compliant'] = True
        self.df[self.df[sum_column] < k, 'K_Anonymity_Compliant'] = False
        
        
    def l_diversity(self, organization_columns, l):
        self.df['L_Diversity_Compliant_One'] = True
        grouped_df = self.df[self.df['K_Anonymity_Compliant'] == True]
        valid_groups = diversity_count[diversity_count['l_diversity'] >= l]
        
        self.df = pd.merge(self.df, valid_groups[quasi_identifiers], on=quasi_identifiers, how='left', indicator='L_Diversity_Compliant')
        self.df['L_Diversity_Compliant'] = self.df['L_Diversity_Compliant'] == 'both'

    def t_closeness(self, organization_columns, t=0.2):
        total_count = self.df.count()
        overall_dist = self.df.groupBy(sensitive_columns).count().withColumn('global_prob', F.col('count') / total_count)
        
    def less_than_threshold_not_zero(self, frequency, minimum_threshold=10):
        mask = (self.df[frequency] < minimum_threshold) & (self.df[frequency] != 0)
        self.df.loc[mask, 'Redact'] = f'Less Than {minimum_threshold} and not zero' 
        return self.df

    def sum_redact (self, frequency, organization_colummns=None, minimum_threshold=10):
        # Sub-query for selecting records where frequency < minimum_threshold and != 0, and then group by group_culumns
        df_less_than = self.df[(self.df[frequency] < minimum_threshold) & (self.df[frequency] != 0)]
        df_grouped_less_than = df_less_than.groupby(organization_columns)[target_variable].sum().reset_index(name='TotalCount')
        df_filtered_group = df_grouped_less_than[df_grouped_less_than['TotalCount'] <= minimum_threshold]
        df_filtered = df_filtered_group[organization_columns]
        
        # Sub-query for selecting the minimum StudentCount >= 11 for each DimSeaID
        df_greater_than = self.df[self.df[frequency] >= minimum_threshold]
        df_grouped_greater_than = df_greater_than.groupby(organization_columns)[frequency].min().reset_index(name='MinimumValue')
        
        df_result = pd.merge(df_filtered, df_grouped_greater_than, on=organization_columns, how='inner')
        
        greater_than_columns = organization_columns + ['MinimumValue']

        merged_columns = organization_columns + [frequency]
        
        # Joining the first and second subqueries based on DimSeaID
        self.df = pd.merge(self.df, df_result, left_on=merged_columns, right_on=greater_than_columns, how='left')
        
        self.df.loc[~self.df['MinimumValue'].isnull(), 'Redact'] = 'Sum of minimum threshold redact needed secondary suppression'

        self.df = self.df[organization_columns + sensitive_columns + [frequency] + ['Redact']]
        return self.df
        
    def one_count_redacted (self, frequency, organization_colummns=None, minimum_threshold=10):
        # Inner query: Group by DimSeaID where StudentCount < 11 and != 0, and count the StudentCount values
        df_less_than_eleven_count = self.df[(self.df[frequency] < threshold) & (self.df[frequency] != 0)]
        df_grouped = df_less_than_eleven_count.groupby(organization_columns).size().reset_index(name='counts')
        df_filtered_grouped = df_grouped[df_grouped['counts'] == 1]
        df_filtered = df_filtered_grouped[organization_columns]
        display(df_filtered)
        
        # Inner join subquery: Find the minimum StudentCount that is at least 11, grouped by DimSeaID
        df_minimum_threshold = self.df[self.df[frequency] >= threshold]
        df_grouped_min = df_minimum_threshold.groupby(organization_columns)[frequency].min().reset_index(name='MinimumValue')

        df_result = pd.merge(df_filtered, df_grouped_min, on=organization_columns, how='inner')


        greater_than_columns = organization_columns + ['MinimumValue']

        merged_columns = organization_columns + [frequency]
        
        # Final query: Join the two DataFrame pieces on DimSeaID
        self.df = pd.merge(self.df, df_result, left_on=merged_columns, right_on=greater_than_columns, how='left')

        self.df.loc[~self.df['MinimumValue'].isnull(), 'Redact'] = 'Count minimum threshold needed secondary suppression'

        self.df = self.df[organization_columns + sensitive_columns + [frequency] + ['Redact']]
        return self.df
        
    def redact_threshold(self, frequency, organization_colummns=None, minimum_threshold=10):
        self.less_than_threshold_not_zero(frequency, minimum_threshold)
        self.df['Overlapping'] = 0
        for sensitive_column in self.sensitive_columns:
            list_sensitive = self.df[~self.df['Redact'].isnull()][sensitive_column].unique()
            self.df.loc[self.df[sensitive_column].isin(list_sensitive), 'Overlapping'] = self.df['Overlapping'] + 1
        self.df.loc[self.df['Overlapping'] == self.df['Overlapping'].max(), 'Suppressed'] = 1
        mask = ((self.df['Overlapping'] == self.df['Overlapping'].max()) & (self.df['Redact'].isnull()))
        self.df.loc[mask, 'Redact'] = f'Overlapping threshold secondary suppression' 
        if organization_colummns == None:
            print('No organization columns')
            self.df = self.df[self.sensitive_columns + [frequency] + ['Redact']]
        elif type(organization_colummns) == str:
            organization_colummns = [organization_colummns]
            self.df = self.df[organization_colummns + self.sensitive_columns + [frequency] + ['Redact']]
        else:
            self.df = self.df[organization_colummns + self.sensitive_columns + [frequency] + ['Redact']]
        return self.df

# Edge Case

In [14]:
df2 = pd.read_csv('DataDegreeBeth_edgecase.csv') #Only has one value in one Subgroup combination that needs redaction 
display(df2)
anonymizer = DataAnonymizer(df2, sensitive_columns=['Subgroup1', 'Subgroup2'])
df_redact_edgecase = anonymizer.redact_threshold('Counts')
display(df_redact_edgecase) #Results in under-redaction because there are no overlapping categories 

,Subgroup1,Subgroup2,Counts
0,STEM,Certificate,10
1,STEM,Associate,9
2,STEM,Bachelor,20
3,STEM,Masters,100
4,STEM,Doctorate,40
5,Business,Certificate,15
6,Business,Associate,40
7,Business,Bachelor,15
8,Business,Masters,90
9,Business,Doctorate,11


No organization columns


,Subgroup1,Subgroup2,Counts,Redact
0,STEM,Certificate,10,NaN
1,STEM,Associate,9,Less Than 10 and not zero
2,STEM,Bachelor,20,NaN
3,STEM,Masters,100,NaN
4,STEM,Doctorate,40,NaN
5,Business,Certificate,15,NaN
6,Business,Associate,40,NaN
7,Business,Bachelor,15,NaN
8,Business,Masters,90,NaN
9,Business,Doctorate,11,NaN


In [6]:
df_data = pd.read_csv("DataDegreeBeth.csv")
display(df_data)
# Instantiate the anonymizer
anonymizer = DataAnonymizer(df_data, sensitive_columns=['Subgroup1', 'Subgroup2'])
# Apply Less Than Value Not Zero
df_redact_threshold = anonymizer.redact_threshold('Counts')

display(df_redact_threshold)

,Subgroup1,Subgroup2,Counts
0,STEM,Certificate,10
1,STEM,Associate,9
2,STEM,Bachelor,20
3,STEM,Masters,100
4,STEM,Doctorate,40
5,Business,Certificate,15
6,Business,Associate,40
7,Business,Bachelor,15
8,Business,Masters,90
9,Business,Doctorate,11


No organization columns


,Subgroup1,Subgroup2,Counts,Redact
0,STEM,Certificate,10,NaN
1,STEM,Associate,9,Less Than 10 and not zero
2,STEM,Bachelor,20,NaN
3,STEM,Masters,100,Overlapping threshold secondary suppression
4,STEM,Doctorate,40,NaN
5,Business,Certificate,15,NaN
6,Business,Associate,40,NaN
7,Business,Bachelor,15,NaN
8,Business,Masters,90,NaN
9,Business,Doctorate,11,NaN


In [15]:
df_data = pd.read_csv("DataDegreeBeth.csv")
display(df_data)

,Subgroup1,Subgroup2,Counts,UnderSuppress,OverSuppressed
0,STEM,Certificate,10,0,1
1,STEM,Associate,9,1,1
2,STEM,Bachelor,20,0,0
3,STEM,Masters,100,1,0
4,STEM,Doctorate,40,0,0
5,Business,Certificate,15,0,1
6,Business,Associate,40,0,0
7,Business,Bachelor,15,0,1
8,Business,Masters,90,0,0
9,Business,Doctorate,11,0,1


In [6]:
# Sample DataFrame with 'Organization', 'Parent_Organization', and 'Gender'
data = {
    'Parent_Organization': ['Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent1', 'Parent2', 'Parent2', 'Parent2', 'Parent2', 'Parent2', 'Parent2'],
    'Organization': ['Org1', 'Org1', 'Org1', 'Org2', 'Org2', 'Org2', 'Org3', 'Org3', 'Org3', 'Org4', 'Org4', 'Org4'],
    'Gender': ['F', 'M', 'NB', 'F', 'M', 'NB', 'F', 'M', 'NB', 'F', 'M', 'NB'],
    'GraduationCount': [20, 0, 2, 13, 5, 4, 25, 13, 7, 10, 0, 3]
}

df = pd.DataFrame(data)

df = pd.read_csv('SampleData.csv')

# Instantiate the anonymizer
anonymizer = DataAnonymizer(df)

# Apply Less Than Value Not Zero
anonymizer.less_than_value_not_zero(11, 'GraduationCount')

# Sum of Suppressed Values
anonymizer.sum_suppressed(['ParentEntity', 'ChildEntity'], ['Subgroup'], 11, 'GraduationCount')

# One Count of Suppressed Values
anonymizer.one_count_suppressed(['ParentEntity', 'ChildEntity'], ['Subgroup'], 11, 'GraduationCount')

# Apply Less Than Value Not Zero
anonymizer.one_suppressed_zero(['ParentEntity', 'ChildEntity'], ['Subgroup'], 'GraduationCount')

display(anonymizer.df)

Sensitive Column is empty


AttributeError: 'DataAnonymizer' object has no attribute 'less_than_value_not_zero'